# Helmet Classification For TinyML Project

> 이 notebook 은 open source 컨트리뷰톤 2020 - tinyML (Tensorflow Lite Project) Mobility Team 의 오픈소스 프로젝트를 위해 만들어졌습니다. 

- 모빌리티 팀 (멘토 맹윤호)
- 최예진(팀장), 이민우, 전수민, 이장후, 이경환, 조승현
- **.ipynb 제작 - 이장후. 2020/08/29**
- **.ipynb 수정자 -**

<br>

- Target Github Repository : [TinyML : Tensorflow lite for microcontroller](https://github.com/yunho0130/tensorflow-lite)
- Team Github Repository : [TinyML-Mobility](https://github.com/orgs/tinyml-mobility/teams)

<br>

## Before We Start
- 런타임 -> GPU 로 변경 하셨나요?

<br>

## This Time
- 생성된 h5 모델을 불러들여 tflite 파일로 변환해 봅시다.
- 구현된 모델을 조금 수정하면서, Class Activation Map* 을 한번 visualization 해 봅시다.

*Class Activation Map 이란, Helmet 클래스로 판단하는 데 어떤 부분을 가장 주목해서 보았는지와 같이, 어떤 클래스로 판단하는 것의 근거를 Visualization 한 이미지를 의미합니다. 


# Google Drive

- 학습을 시키기 전 데이터가 있는 google Drive 와 연동을 해야 합니다.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

## Include Library

- 이 노트북의 소스코드는 tensorflow 2.0 이상과 호환되지 않습니다.
- Google colab 에서는 %tensorflow_version 을 통해, 원하는 버전의 tensorflow 를 쉽게 불러올 수 있습니다

In [ ]:
try:
  # This %tensorflow_version magic only works in Colab.
  %tensorflow_version 1.x
except Exception:
  pass

# For your non-Colab code, be sure you have tensorflow==1.15
import tensorflow as tf
assert tf.__version__.startswith('1')

# tensorflow 는 기본적으로, "정적 그래프 형식" 으로 실행하여 그때그때 실행해서 결과를 찍어보는 것이 불가능합니다.
# tf.enable_eager_execution 을 실행해 주어야, datagen 으로 실행이 가능합니다.
# 이 코드는, 
tf.enable_eager_execution()


import os
import numpy as np
import matplotlib.pyplot as plt



In [ ]:
tf.__version__

# Overview

## TinyML
- 모델을 .tflite 파일로 변환하기 위해 tensorflow lite 의 python API 를 활용할 것입니다.
- .tflite 는 플랫버퍼 형식이라고 합니다.
- 플랫버퍼 형식에 대한 장점은 굉장히 많다고 하지만, 저를 포함해서 이 튜토리얼을 진행하는 분들에게는 너무 어려운 내용일 것입니다. 그냥 "효율적인 자료 저장 형식이다" 라고 생각합시다!


## Optimization
- 우리는 지금 "작은 모델을 만들기 위해" part3 으로 넘어왔습니다.
- 우리가 다음 작업을 진행하기 전에, 확인해야 할 큰 그림이 있습니다. 이 내용은 tinyML 책 챕터 15에 자세히 나와 있습니다. 이를 간단히 이야기해보도록 해요.

<br>

**Hardware Selection**
- 주머니 사정과 성능, 접근성 및 개발 속도를 모두 고려하여 하드웨어를 선택해야 합니다.
- 저희는 이 프로젝트를 진행하며, 파일을 Raspberry Pi 4에 업로드할 것입니다.
- Raspberry Pi 4 는 단돈 5만원에 매우 강력한 성능을 자랑하는 컴퓨터로, 비영리 재단에서 만든 소형 컴퓨터입니다.
- Raspberry Pi 4 는 일반적인 마이크로컨트롤러들과 달리, 운영체제가 올라가고 메모리와 디스크 모두 넉넉합니다.
- Raspberry Pi 4 는 다양한 주변기기를 연결할 수 있도록 지원하고, 저희는 종내에 소형 GPU 를 사용해서 모델을 돌려 볼 수 있도록 할 것입니다.

<br>

**Model Selection**
- 우리는 헬멧 인식 모델을 만들 것이고, 헬멧을 착용하기 위한 모델은 굉장히 복잡합니다.
- 소형 기기에 배포하기 위해 이 모델을 가볍게 만드는 것은 전력 소비 / 실행 속도 / 이용자 체감 에서도 매우 중요한 이슈겠지요.
- 우리는 tinyML 책에서 제시된 MobileNet v1 보다 효율적인 MobileNet v2 를 사용했습니다.
- 일반적인 CNN 모델들보다 훨씬 효율적인 모델입니다.

<br>

**Quantization**
- 많은 임베디드 디바이스에서는...
- 오늘 할 작업입니다!





In [ ]:
%cd /content/gdrive/"My Drive"/data/

# 데이터가 존재하는 경로 ( /content/gdrive/"My Drive"/data/helmetclassification ) 를 data_dir 변수에 저장합니다.
data_dir = os.path.join(os.getcwd(), 'helmetclassification')
print(data_dir)

# input 이미지의 크기는 160 by 160 by 3 으로 상정합니다. 채널은 RGB 이므로, 3 입니다.
IMG_WIDTH = 224
IMG_HEIGHT = 224
IMG_CHANNEL = 3
IMG_SHAPE = (IMG_WIDTH, IMG_HEIGHT, IMG_CHANNEL)

# 연산 처리 단위 (배치) 는 이미지 16장, 그리고 Learning Rate, optimizer, 에폭 등을 설정합니다.
# *참고* : Optimizer 에서 SGD 는 최근 잘 사용하지 않지만, 안정적인 수렴을 위해 특정 경우에 사용합니다. 
BATCH_SIZE = 16
LEARNING_RATE_SGD = 0.001
LEARNING_RATE_ADAM = 0.0001
TRAINING_OPTIMIZER_SGD  = tf.keras.optimizers.SGD(learning_rate=LEARNING_RATE_SGD, momentum=0.0)
TRAINING_OPTIMIZER_ADAM = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE_ADAM)
EPOCHS = 50

# 우리는 checkpoint 를 설정해서, 각 epoch 마다 가중치를 저장할 것입니다.
CKPT_DIR = os.path.join(data_dir, 'checkpoint')

# 우리는 h5 file 을 tflite 파일로 전환할 것입니다. 이름을 미리 정해 둡시다.
SAVED_KERAS_MODEL_NAME = 'helmet_classification_model.h5'

# 이번에 우리는 h5 file 을 읽어와 tflite 파일로 바꾸어 저장할 것입니다. 이름을 미리 정해 둡시다.
SAVED_TFLITE_MODEL_NAME = 'helmet_classification_model.tflite'

# 우리는 잠시 후에 라벨 파일을 만들어낼 것인데, 라벨 파일의 이름을 미리 정의해 둡시다.
LABEL_FILE_NAME = 'ishelmetlabel.txt'

# Quantization

## Idea
- 어떤 작업인지는 위에서 설명...
- 학습한 데이터셋의 입력값 범위를 나타내는 숫자의 집합인 대표 예시 데이터셋을 만들어 넣어 주어야 합니다..
- 그 이유는...

## How to
- 예시 데이터를 만들어 줄 수 있는 generator

In [ ]:
%cd /content/gdrive/"My Drive"/data/helmetclassification

converter = tf.lite.TFLiteConverter.from_keras_model_file(SAVED_KERAS_MODEL_NAME)

In [ ]:
# 양자화를 포함해서, 기본 최적화를 진행합니다.
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# 예시 데이터를 가져다주어야 한다고 했습니다.

def representative_data_gen():
  dataset_list = tf.data.Dataset.list_files(data_dir + '/test' + '/*/*')
  print('dataset_list : ', dataset_list)
  for i in range(100):
    image = next(iter(dataset_list))
    image = tf.io.read_file(image)
    image = tf.io.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [IMG_WIDTH, IMG_HEIGHT])
    image = tf.cast(image / 255., tf.float32)
    image = tf.expand_dims(image, 0)
    yield [image]

im = representative_data_gen()

print(next(im))

# converter 객체에 등록해 줍니다.
converter.representative_dataset = representative_data_gen


# 실제값 = (int8변환값 - 영점) * scale
# int8변환값 = 실제값 / scale + 영점
# input 은 float 이든 int 이든, 어차피 255 배 커지는건데 뭔상관?!
# 255배 크게 활성화되든 작게 활성화되든 이미 training 은 끝났거든~~
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

# 아래 주석처리된 소스코드는, INT8 로 전환할 때, 양자화를 지원하지 않는 operation 이 있는지 없는지 확인해 볼 수 있습니다.
# 원래 CORAL 과 같은 int8 자료형만을 지원하는 디바이스에 업로드하기 위해서는
# 아래 코드를 활성화해야 하지만, 
# 아직 mobilenet 의 모든 기능을 구현할 수 있도록 operation 이 support 되지 않는 모양입니다. 
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

In [ ]:
# 변환된 객체를 tflite_model 변수에 저장합니다.
tflite_model = converter.convert()

In [ ]:
%cd /content/gdrive/"My Drive"/data/helmetclassification

# 파일을 저장합니다.
open(SAVED_TFLITE_MODEL_NAME, "wb").write(tflite_model)

# Class Activation Mapping
